In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/ZUM/Project 1/Основний варіант/')

print(os.listdir('/content/drive/My Drive/ZUM/Project 1/Основний варіант/'))

import pandas as pd

Mounted at /content/drive
['anime_no_index.csv', 'anime', 'war', 'task2_1.ipynb', 'task_1_2_word2vec.model.wv.vectors.npy', 'task_1_2_word2vec.model.syn1neg.npy', 'task_1_2_word2vec.model', 'task_1_2_sentiment_dictionary.csv', 'task1_2.ipynb', 'task1_1.ipynb', 'task_1_1_result.csv', 'task_1_1_result.gsheet']


In [2]:

df = pd.read_csv('task_1_1_result.csv')

df.sample(20)

,Tweet
7482,japn intentando hacer un anime donde haya ning...
49648,kaidotio zoro abre o olho esquerdo e robin pis...
87461,try tell be not good anime history
214951,blind box toy original sank road journey serie...
141127,tesseractive live 38 bless year fully know hom...
32224,directedbysyle berserknoxy lanime ne mimpacte ...
128434,shunksanime greentextrepost genuinely matter p...
123632,4kingdomsttrpg character trope feardislike 1 l...
2878,can not beat ass world cuz do not exist world ...
114494,offense anime tiddie biggg like gigantic monst...


In [3]:
words = []

for i in df['Tweet'].values:
  words.append(str(i).split())


In [4]:
len(words)

216385

# Word2Vec

In [5]:
import multiprocessing
from time import time 
from gensim.models import Word2Vec

w2v_model = Word2Vec(min_count=3,
                     window=4,
                     vector_size=300,
                     sample=1e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=multiprocessing.cpu_count()-1)

start = time()

w2v_model.build_vocab(words, progress_per=50000)

print('Time to build vocab: {} mins'.format(round((time() - start) / 60, 2)))

Time to build vocab: 0.04 mins


In [6]:
start = time()

w2v_model.train(words,
                total_examples=w2v_model.corpus_count,
                epochs=30,
                report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - start) / 60, 2)))

w2v_model.init_sims(replace=True)

<ipython-input-6-70a3ae6b16e9>:10: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v_model.init_sims(replace=True)


Time to train the model: 6.57 mins


In [7]:
w2v_model.save("task_1_2_word2vec.model")

# KMeans cluster

In [8]:
from sklearn.cluster import KMeans
word_vectors = Word2Vec.load("task_1_2_word2vec.model").wv

model = KMeans(n_clusters=2,
               max_iter=1000,
               random_state=42,
               n_init=50)
model.fit(X=word_vectors.vectors.astype('double'))

KMeans(max_iter=1000, n_clusters=2, n_init=50, random_state=42)

In [9]:
word_vectors.similar_by_vector(model.cluster_centers_[1], topn=20, restrict_vocab=None)

[('py6kt', 0.9830756187438965),
 ('gotham66', 0.9794917106628418),
 ('autixcx', 0.9782661199569702),
 ('theeuropeanlad', 0.974713146686554),
 ('tid', 0.9744638800621033),
 ('mormondruguser', 0.9735771417617798),
 ('xphysixal', 0.9731224775314331),
 ('paneknwym', 0.9720859527587891),
 ('beangoblinvt', 0.9719280004501343),
 ('charlot97161061', 0.9716536998748779),
 ('spuuky', 0.971340537071228),
 ('woop', 0.9713160991668701),
 ('dyl', 0.971192479133606),
 ('angellucywrite', 0.9710063934326172),
 ('rcbtweet', 0.9709203839302063),
 ('discolitaa', 0.9708033800125122),
 ('machographix', 0.9705963134765625),
 ('endikux', 0.9698764085769653),
 ('easternooc', 0.9694362282752991),
 ('hru', 0.9686556458473206)]

In [10]:
positive_cluster_index = 1
positive_cluster_center = model.cluster_centers_[positive_cluster_index]
negative_cluster_center = model.cluster_centers_[1-positive_cluster_index]

In [11]:
import numpy as np

words = pd.DataFrame(word_vectors.index_to_key)
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])
words['cluster_value'] = [1 if i==positive_cluster_index else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [12]:
words.sample(50)

,words,vectors,cluster,cluster_value,closeness_score,sentiment_coeff
22640,independente,"[-0.004361275, -0.012531305, 0.08497636, 0.054...",0,-1,1.442855,-1.442855
36801,junhui,"[-0.036696546, 0.021024209, -0.021682564, 0.14...",1,1,1.369277,1.369277
11046,animeuniver,"[0.048580512, 0.03244392, -0.047652222, 0.1286...",0,-1,1.123995,-1.123995
55432,maouzxchkq,"[-0.014639547, 0.115847, -0.028644638, 0.09221...",1,1,1.498808,1.498808
5214,smell,"[-0.056403313, 0.134638, -0.04199815, 0.044688...",1,1,1.119362,1.119362
56614,perfekte,"[0.023306334, 0.1275158, -0.008857199, 0.11353...",1,1,1.343494,1.343494
1216,here,"[-0.07669101, 0.039719887, -0.03352958, 0.0149...",1,1,1.097458,1.097458
39776,saveur,"[0.06286272, 0.038939837, 0.064647146, 0.08976...",1,1,1.357723,1.357723
42543,muncrat,"[0.017174363, -0.015135303, 0.03725835, 0.1823...",1,1,1.200259,1.200259
52484,readin,"[-0.034647815, 0.13826656, 0.01252218, 0.09697...",1,1,1.728822,1.728822


In [13]:
words[['words', 'sentiment_coeff']].to_csv('task_1_2_sentiment_dictionary.csv', index=False)

# Labelling data

In [14]:
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [15]:
final_file = pd.read_csv('task_1_1_result.csv')
sentiment_map = pd.read_csv('task_1_2_sentiment_dictionary.csv')
sentiment_dict = dict(zip(sentiment_map.words.values, sentiment_map.sentiment_coeff.values))

In [16]:
def test_print(y):
  return y.split()

In [17]:
file_weighting = final_file.copy()
tfidf = TfidfVectorizer(tokenizer=lambda y: test_print(y), norm=None)
tfidf.fit(file_weighting.values.ravel())
features = pd.Series(tfidf.get_feature_names_out())
transformed = tfidf.transform(file_weighting.values.ravel())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [21]:
def create_tfidf_dictionary(x, i, transformed_file, features):
    vector_coo = transformed_file[i].tocoo()
    vector_coo.col = features.iloc[vector_coo.col].values
    dict_from_coo = dict(zip(vector_coo.col, vector_coo.data))
    return dict_from_coo

def replace_tfidf_words(x, i, transformed_file, features):
    dictionary = create_tfidf_dictionary(x, i, transformed_file, features)  
    #print(((lambda y:dictionary[f'{y}'], x.split()))) 
    #print(list(map(lambda y:dictionary[f'{y}'], x.split())))
    return list(map(lambda y:dictionary[f'{y}'], x.split()))

In [ ]:
replaced_tfidf_scores = []

for i in range(len(file_weighting)):
  #print(i)
  #replace_tfidf_words(file_weighting['Tweet'][i], i, transformed, features)
  replaced_tfidf_scores.append(replace_tfidf_words(file_weighting['Tweet'][i], i, transformed, features))

In [ ]:
def replace_sentiment_words(word, sentiment_dict):
    try:
        out = sentiment_dict[word]
    except KeyError:
        out = 0
    return out

In [ ]:
replaced_closeness_scores = file_weighting['Tweet'].apply(lambda x: list(map(lambda y: replace_sentiment_words(y, sentiment_dict), x.split())))

In [ ]:
print(len(replaced_tfidf_scores))
print(len(file_weighting))
print(len(replaced_closeness_scores))

In [ ]:
import numpy as np

replacement_df = file_weighting.copy()
replacement_df['sentiment_coeff'] = replaced_closeness_scores
replacement_df['tfidf_scores'] = replaced_tfidf_scores

replacement_df['sentiment_rate'] = replacement_df.apply(lambda x: np.array(x.loc['sentiment_coeff']) @ np.array(x.loc['tfidf_scores']), axis=1)
replacement_df['prediction'] = (replacement_df.sentiment_rate>0).astype('int8')

In [ ]:
replacement_df.sample(100)

In [ ]:
replacement_df[['Tweet', 'prediction']].to_csv("task_1_2_labeled_data.csv", index=False)